<a href="https://colab.research.google.com/github/madan-genai/1-Review-Sentiment-Analysis-RNN/blob/main/Tools_In_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install langchain langchain_community langchain-core pydantic duckduckgo-search

In [6]:
!pip install -U ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 15.2 MB/s eta 0:00:00


Built-in-Tool- DuckDuckGo Search

In [8]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke("PSL news")
results

"1 day ago - The 2026 Pakistan Super League (PSL 11) will be the eleventh season of the Pakistan Super League , a franchise Twenty20 cricket tournament organised by the Pakistan Cricket Board (PCB). The season will mark the league's expansion from six to eight franchises, following the PCB's confirmation ... 2 days ago - However, scheduling changes announced by the Pakistan Cricket Board in 2025 confirmed that from the 2026 season onward, the PSL would move to an April–May window to improve overseas player availability and avoid weather disruptions . Declaración del PSL: ¡No a la guerra contra Venezuela – rechazamos el secuestro de Nicolás Maduro! 12 days agoFBI tip targets PSL man who 'expressed sympathy for mass shooters' Published 4 days ago 1 week ago - Pakistan Super League (PSL) will move to a player auction model from its 11th season , replacing the draft system that has been in place since the league's inception."

Built-in-Tool-Shell Tool

In [10]:
!pip install langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 16.6 MB/s eta 0:00:00


In [12]:
from langchain_community.tools import  ShellTool

shell_tool = ShellTool()
results = shell_tool.invoke("ls")
results

Executing command:
 ls


'sample_data\n'

Custom Tools

In [13]:
from langchain_core.tools import tool

In [14]:
# Step1 -create a function
def multiply (a,b):
  """Multiply two numbers"""
  return a*b


In [15]:
# Step 2 - add type hints
def multiply(a: int, b:int) -> int:
  """Multiply two numbers"""
  return a*b

In [16]:
#Step3 - add tool decorator
@tool
def multiply(a:int, b:int) -> int:
  """Multiply two numbers"""
  return a*b


In [17]:
result = multiply.invoke({"a":2,"b":3})


In [18]:
print(result)

6


In [19]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [20]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


# Method 2 - Using Structured Tool

In [44]:
from langchain_community.tools import StructuredTool
from pydantic import BaseModel, Field

In [53]:
class MultiplyInput(BaseModel):
  a: int = Field(..., description="The First number to add")
  b : int = Field(..., description="The Second number to add")

In [54]:
def multiply_func(a:int, b:int) -> int:
  return a*b

In [55]:
multiply_func=StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

In [56]:
result = multiply_func.invoke({"a":4,"b":5})
print(result)

print(multiply_func.name)
print(multiply_func.description)
print(multiply_func.description)

20
multiply
Multiply two numbers
Multiply two numbers


# Method 3- Using BaseTool Class

In [91]:
from langchain.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field

In [92]:
# arg schema using pydantic
class MultiplyInput(BaseModel):
  a:int = Field(..., description="The first number to add")
  b:int = Field(..., description="The second number to add")


In [93]:
class MultiplyTool(BaseModel):
  name : str ="Multiply"
  description: str="multiply two numbers"
  args_schema: Type[BaseModel] = MultiplyInput

  def _run(self, a:int, b:int) ->int:
    return a * b
  async def _arun(self, a: int, b: int) -> int:
        return a + b

In [94]:
multiply_tool = MultiplyTool()
print(multiply_tool.run({ "a":3,"b":5}))

AttributeError: 'MultiplyTool' object has no attribute 'run'

In [89]:
result = multiply_tool.run({"a":3,"b":4})
print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args_schema)

AttributeError: 'MultiplyTool' object has no attribute 'run'

In [70]:
result = multiply_func.invoke({"a":3,"b":4})
print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args_schema)

12
Multiply
multiply two numbers
<class '__main__.MultiplyInput'>


In [82]:
from langchain.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field


In [83]:
class AddInput(BaseModel):
    a: int = Field(description="First number")
    b: int = Field(description="Second number")


In [84]:
class AddTool(BaseTool):
    name: str = "add_numbers"
    description: str = "Add two numbers together"
    args_schema: Type[BaseModel] = AddInput

    def _run(self, a: int, b: int) -> int:
        return a + b

    async def _arun(self, a: int, b: int) -> int:
        return a + b


In [85]:
tool = AddTool()
print(tool.run({"a": 5, "b": 7}))


12


In [95]:
from langchain.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field

In [96]:
class MultiplyInput(BaseModel):
  a: int = Field(description="first number")
  b: int = Field(description="second number")


In [97]:
class MultiplyTool(BaseModel):
  name : str = "multiply numbers"
  description : str ="Multiply two numbers"
  args_schema : Type[BaseModel] = MultiplyInput

  def _run(self, a:int, b:int) -> int:
    return a * b


In [98]:
multiply_tool=MultiplyTool()

In [101]:
print(multiply_tool.invoke({"a":3,"b":5}))


AttributeError: 'MultiplyTool' object has no attribute 'invoke'

# ToolKit

In [103]:
from langchain_core.tools import tool
@tool
def add(a:int, b:int) ->int:
  """Add two numbers"""
  return a + b
@tool
def multiply(a:int, b:int)->int:
  """Multiply two numbers"""
  return a*b


In [104]:
class MathToolkit:
  def get_tools(self):
    return[add,multiply]

In [105]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
  print(tool.name, "=>", tool.description)

add => Add two numbers
multiply => Multiply two numbers
